In [108]:
import tensorflow as tf

In [109]:
daily_sales_numbers = [21,22,-108,31,-1,32,34,31]

In [110]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)

In [111]:
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [112]:
for sales in tf_dataset:
    print(sales)

tf.Tensor(21, shape=(), dtype=int32)
tf.Tensor(22, shape=(), dtype=int32)
tf.Tensor(-108, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)
tf.Tensor(-1, shape=(), dtype=int32)
tf.Tensor(32, shape=(), dtype=int32)
tf.Tensor(34, shape=(), dtype=int32)
tf.Tensor(31, shape=(), dtype=int32)


In [113]:
for sales in tf_dataset:
    print(sales.numpy())

21
22
-108
31
-1
32
34
31


In [114]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
-108
31
-1
32
34
31


In [115]:
for sales in tf_dataset.take(3):
    print(sales.numpy())

21
22
-108


In [116]:
tf_dataset = tf_dataset.filter(lambda x:x>0)

In [117]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

21
22
31
32
34
31


In [118]:
tf_dataset = tf_dataset.map(lambda x:x*72)

In [119]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
1584
2232
2304
2448
2232


In [120]:
tf_dataset = tf_dataset.shuffle(3)
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

1512
2304
2448
2232
1584
2232


In [121]:
for sales_batch in tf_dataset.batch(4):
    print(sales_batch.numpy())

[1584 2232 2448 2232]
[1512 2304]


In [122]:
tf_dataset = tf.data.Dataset.from_tensor_slices(daily_sales_numbers)
tf_dataset

<TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [123]:
tf_dataset = tf_dataset.filter(lambda x:x>0).map(lambda x:x*72).shuffle(3).batch(4)

In [124]:
for sales in tf_dataset.as_numpy_iterator():
    print(sales)

[1512 2232 2448 2232]
[1584 2304]


In [125]:
images_ds = tf.data.Dataset.list_files('images/*/*.jpg', shuffle=False)

for file in images_ds.take(5):
    print(file.numpy())

b'images\\cat\\1500_ Baby Cat Pictures _ Download Free.jpg'
b'images\\cat\\20_ Cat Pictures & Images [HD (1).jpg'
b'images\\cat\\20_ Cat Pictures & Images [HD (2).jpg'
b'images\\cat\\20_ Cat Pictures & Images [HD (3).jpg'
b'images\\cat\\20_ Cat Pictures & Images [HD.jpg'


In [126]:
images_ds = images_ds.shuffle(10)

for file in images_ds.take(3):
    print(file.numpy())

b'images\\cat\\40_000_ Cat Pictures & Images [HD (2).jpg'
b'images\\cat\\20_ Cat Pictures & Images [HD (1).jpg'
b'images\\cat\\40_000_ Cat Pictures & Images [HD.jpg'


In [127]:
class_names = ['cat','dog']

In [128]:
image_count = len(images_ds)
image_count

63

In [129]:
train_size = int(image_count * 0.8)
train_size

50

In [130]:
train_ds = images_ds.take(train_size)
test_ds = images_ds.skip(train_size)

In [131]:
print(len(train_ds))
print(len(test_ds))

50
13


In [132]:
s = 'images\\cat\\20_ Cat Pictures & Images [HD.jpg'

s.split("\\")[1]

'cat'

In [133]:
def get_label(file_path):
    import os
    return tf.strings.split(file_path,os.path.sep)[1]

In [134]:
def process_image(file_path):
    label = get_label(file_path)
    
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img)
    img = tf.image.resize(img,[128,128])
    
    return img,label

In [135]:
for t in train_ds.take(4):
    print(t.numpy())

b'images\\cat\\20_ Cat Pictures & Images [HD (2).jpg'
b'images\\cat\\40_000_ Cat Pictures & Images [HD.jpg'
b'images\\cat\\20_ Cat Pictures & Images [HD (3).jpg'
b'images\\cat\\1500_ Baby Cat Pictures _ Download Free.jpg'


In [136]:
train_ds = train_ds.map(process_image)
for img,label in train_ds.take(3):
    print("Image : ", img)
    print("Label : ",label)

Image :  tf.Tensor(
[[[254.      254.      254.     ]
  [254.      254.      254.     ]
  [254.      254.      254.     ]
  ...
  [254.      254.      254.     ]
  [254.      254.      254.     ]
  [254.      254.      254.     ]]

 [[254.      254.      254.     ]
  [254.      254.      254.     ]
  [254.      254.      254.     ]
  ...
  [254.      254.      254.     ]
  [254.      254.      254.     ]
  [254.      254.      254.     ]]

 [[254.      254.      254.     ]
  [254.      254.      254.     ]
  [254.      254.      254.     ]
  ...
  [254.      254.      254.     ]
  [254.      254.      254.     ]
  [254.      254.      254.     ]]

 ...

 [[162.26721 162.26721 162.26721]
  [156.65625 156.65625 156.65625]
  [155.5567  155.5567  155.5567 ]
  ...
  [135.11578 135.11578 135.11578]
  [144.63757 144.63757 144.63757]
  [149.16449 149.16449 149.16449]]

 [[164.72986 164.72986 164.72986]
  [162.52655 162.52655 162.52655]
  [161.33636 161.33636 161.33636]
  ...
  [136.95654 136.9

In [137]:
def scale(image,label):
    return image/255,label

In [139]:
train_ds = train_ds.map(scale)
for image,label in train_ds.take(5):
    print("*****Image : ",image.numpy()[0][0])
    print("*****Label : ",label.numpy())

*****Image :  [0.34788144 0.39149356 0.35365653]
*****Label :  b'cat'
*****Image :  [0.99607843 0.99607843 0.99607843]
*****Label :  b'cat'
*****Image :  [0.02598709 0.04951651 0.03383023]
*****Label :  b'cat'
*****Image :  [0.77254903 0.7254902  0.6784314 ]
*****Label :  b'cat'
*****Image :  [0.07450981 0.0627451  0.04313726]
*****Label :  b'cat'
